# SQL编程

# psycopg2

Python通过[psycopg2](https://github.com/sourcegraph/psycopg2)访问PostgreSQL数据库实现混合编程，psycopg2的[帮助文档](http://initd.org/psycopg/)，遵循[Python Database API Specification v2.0](https://www.python.org/dev/peps/pep-0249/)

In [ ]:
import psycopg2

psycopg2提供了connection类，提供了connect, cursor, execute, commit, rollback, close等函数。

### 数据库连接

[connect](http://initd.org/psycopg/docs/module.html#psycopg2.connect)函数实现数据库连接：

conn = psycopg2.connect("dbname=test user=postgres password=secret")

conn = psycopg2.connect(dbname="test", user="postgres", password="secret")

cursor函数返回一个[cursor](http://initd.org/psycopg/docs/cursor.html#cursor)类

cursor(name=None, cursor_factory=None, scrollable=None, withhold=False)

### 事务管理

execute函数执行一个SQL语句，一个事务从创建cursor开始或前一个事务commit或rollback开始，以这个事务commit或rollback结束

conn.commit() -- 新事务开始

cur.execute(" SQL ")<br\>
cur.execute(" SQL ")<br\>
check user input<br\>
cur.execute(" SQL ")<br\>

conn.commit()
或
conn.rollback() -- 该事务结束

### 结束连接

结束数据库连接调用conn.close()

通过pgAdmin 4创建Ex13数据库，练习关系创建、数据插入、查询、更新、删除、和事务管理等

In [ ]:
conn = psycopg2.connect(database="Ex13", user="postgres", password="postgres", host="127.0.0.1", port="5432") 
cur = conn.cursor()

cur.execute('''Drop Table if exists Company;''')
cur.execute('''CREATE TABLE COMPANY
              (ID INT PRIMARY KEY     NOT NULL,
               NAME           TEXT    NOT NULL,
               AGE            INT     NOT NULL,
               ADDRESS        CHAR(50),
               SALARY         REAL);''')

conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(database="Ex13", user="postgres", password="postgres", host="127.0.0.1", port="5432") 
cur = conn.cursor()

cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (1, 'Paul', 32, 'California', 20000.00 )");

cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (2, 'Allen', 25, 'Texas', 15000.00 )");

cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (3, 'Teddy', 23, 'Norway', 20000.00 )");

cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (4, 'Mark', 25, 'Rich-Mond ', 65000.00 )");

conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(database="Ex13", user="postgres", password="postgres", host="127.0.0.1", port="5432") 
cur = conn.cursor()

cur.execute("SELECT id, name, address, salary  from COMPANY")
rows = cur.fetchall()
for row in rows:
    print("ID = ", row[0])
    print("NAME = ", row[1])
    print("ADDRESS = ", row[2])
    print("SALARY = ", row[3], "\n")

conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(database="Ex13", user="postgres", password="postgres", host="127.0.0.1", port="5432") 
cur = conn.cursor()

cur.execute("DELETE from COMPANY where ID=2;")
conn.commit()
print("Total number of rows deleted :", cur.rowcount)

cur.execute("UPDATE COMPANY set SALARY = 25000.00 where ID=1")
conn.rollback()
print("Total number of rows updated :", cur.rowcount)

cur.execute("SELECT id, name, address, salary from COMPANY")
rows = cur.fetchall()
for row in rows:
    print("ID = ", row[0])
    print("NAME = ", row[1])
    print("ADDRESS = ", row[2])
    print("SALARY = ", row[3], "\n")

conn.commit()
conn.close()

# C++

SOCI - [The C++ Database Access Library](http://soci.sourceforge.net/doc/master/)

SOCI is a database access library written in C++ that makes an illusion of embedding SQL queries in the regular C++ code, staying entirely within the Standard C++.

The idea is to provide C++ programmers a way to access SQL databases in the most natural and intuitive way. If you find existing libraries too difficult for your needs or just distracting, SOCI can be a good alternative.

<h2>Basic Syntax</h2>
<p>The simplest motivating code example for the SQL query that is supposed to retrieve a single row is:</p>
<pre><code>
session sql(postgresql, "dbname=mydb");

int id = ...;
string name;
int salary;

sql &lt;&lt; &quot;select name, salary from persons where id = &quot; &lt;&lt; id,
        into(name), into(salary);
</code></pre>

## Connection

创建[session](http://soci.sourceforge.net/doc/master/connections/)类，例如

session sql("postgresql://dbname=mydb");

## Prepared Statement

避免SQL注入，可以使用[Prepared Statement](http://soci.sourceforge.net/doc/master/statements/)。

<pre><code>// Example 1.
for (int i = 0; i != 100; ++i) {
    sql &lt;&lt; &quot;insert into numbers(value) values(&quot; &lt;&lt; i &lt;&lt; &quot;)&quot;;
}

// Example 2.
for (int i = 0; i != 100; ++i) {
    sql &lt;&lt; &quot;insert into numbers(value) values(:val)&quot;, use(i);
}
</code></pre>

上述语句可以使用Prepared Statement避免SQL注入
<pre><code>int i;
statement st = (sql.prepare &lt;&lt;
                &quot;insert into numbers(value) values(:val)&quot;,
                use(i));
for (i = 0; i != 100; ++i) {
    st.execute(true);
}
</code></pre>

## Transactions

session类提供了
* void begin();
* void commit();
* void rollback();
实现事务管理。同时也提供了[Transaction](http://soci.sourceforge.net/doc/master/transactions/)管理事务。
<pre><code>
class transaction
{
public:
    explicit transaction(session &amp; sql);

    ~transaction();

    void commit();
    void rollback();
private:
    //...
};
</code></pre>


# React Native

移动端的数据库连接和操作。

TODOList例子，App初始化时，连接sqlite数据库todo.db，如果不存在关系TODOs，创建关系TODOs，查询数据库中的所有task，存储在items，供界面显示。executeSql函数是事务执行一个SQL语句的函数，提供insert, delete, select三类数据操作。

<code>
import React from 'react';

import { Constants, SQLite } from 'expo';

const db = SQLite.openDatabase('todo.db');

export default class App extends React.Component {
  state = {
    textValue: '',
    items: []
  }

  executeSql = async (sql, params = []) => {
    return new Promise((resolve, reject) => db.transaction(tx => {
      tx.executeSql(sql, params, (_, { rows }) => resolve(rows._array), reject)
    }))
  }

  componentWillMount () {
    this.init()
  }

  init = async () => {
    await this.executeSql('create table if not exists TODOs (task);');
    this.select()
  }

  _insert = async () => {
    await this.executeSql('insert into TODOs (task) values (?)', [this.state.textValue]);
    this.setState({textValue: ''});
    return true
  }

  _delete = async (task) => {
    await this.executeSql('delete from TODOs where task=?', [task]);
    return true
  }

  select = () => {
    this.executeSql('select * from TODOs', []).then(items => this.setState({items}));
  }

  insert = () => {
    this._insert().then(this.select)
  }

  deleteRow = (task) => {
    this._delete(task).then(this.select)
  }
}
</code>